# Import

In [2]:
import pandas as pd  # For handling and manipulating data
from sklearn.preprocessing import MinMaxScaler  # For normalizing data
from sklearn.model_selection import train_test_split  # For splitting data into training and validation sets

# Load

In [4]:
train_data = pd.read_csv(r'C:\Users\aksha\Documents\ML_Python\Main\train.csv')
test_data = pd.read_csv(r'C:\Users\aksha\Documents\ML_Python\Main\test.csv')
# drop irrelevant columns
train_data_cleaned = train_data.drop(columns=['id', 'ever_married_No', 'Residence_type_Rural'])
# remove outliers
train_data_cleaned = train_data_cleaned[
    (train_data_cleaned['bmi'] >= 10) & (train_data_cleaned['bmi'] <= 50) &
    (train_data_cleaned['avg_glucose_level'] >= 55) & (train_data_cleaned['avg_glucose_level'] <= 250)
]
from sklearn.preprocessing import MinMaxScaler

# Columns to normalize
columns_to_standardize = ['age', 'bmi', 'avg_glucose_level']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Apply the scaler to the relevant columns
train_data_cleaned[columns_to_standardize] = scaler.fit_transform(train_data_cleaned[columns_to_standardize])
# train test split here
# Defining features and target
X = train_data_cleaned.drop(columns='stroke')
y = train_data_cleaned['stroke']

# Splitting into train and validation sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)

# Test 1

In [ ]:
# Import necessary libraries
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.neural_network import MLPClassifier

# Define Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the parameter grid for GridSearch
param_grid = {
    'hidden_layer_sizes': [(32, 16), (64, 32), (128, 64)],  # Different architectures
    'activation': ['relu', 'tanh'],  # Activation functions
    'solver': ['adam', 'sgd'],       # Optimizers
    'learning_rate_init': [0.001, 0.01],  # Initial learning rates
    'max_iter': [200, 300]           # Training iterations
}

# Define the model
mlp = MLPClassifier(random_state=42)

# Define the F1 scorer
f1_scorer = make_scorer(f1_score)

# Define GridSearchCV
grid_search = GridSearchCV(estimator=mlp, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Perform the grid search
grid_search.fit(X_train, y_train)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score (Train CV): {grid_search.best_score_}")

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print(f"F1 Score on the Test Set: {test_f1:.10f}")


Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 64), 'learning_rate_init': 0.001, 'max_iter': 300, 'solver': 'adam'}
Best F1 Score (Train CV): 0.026060400688542613
F1 Score on the Test Set: 0.0


# Test 2

In [13]:
# Import necessary libraries for balancing
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import make_scorer, f1_score
from sklearn.neural_network import MLPClassifier

# Define the target sampling strategy
smote = SMOTE(random_state=42, sampling_strategy=0.99)
undersample = RandomUnderSampler(sampling_strategy=0.99, random_state=42)  # Target ratio of majority:minority = 0.99

# Combine SMOTE and undersampling in a pipeline
resampling_pipeline = Pipeline([
    ('smote', smote),
    ('undersample', undersample)
])

# Resample the training data
X_train_balanced, y_train_balanced = resampling_pipeline.fit_resample(X_train, y_train)

# Define Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the parameter grid for GridSearch
param_grid = {
    'hidden_layer_sizes': [(32, 16), (64, 32), (128, 64)],  # Different architectures
    'activation': ['relu', 'tanh'],  # Activation functions
    'solver': ['adam', 'sgd'],       # Optimizers
    'learning_rate_init': [0.001, 0.01],  # Initial learning rates
    'max_iter': [200, 300]           # Training iterations
}

# Define the model
mlp = MLPClassifier(random_state=42)

# Define the F1 scorer
f1_scorer = make_scorer(f1_score)

# Define GridSearchCV
grid_search = GridSearchCV(estimator=mlp, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Perform the grid search on the resampled data
grid_search.fit(X_train_balanced, y_train_balanced)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score (Train CV): {grid_search.best_score_}")

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print(f"F1 Score on the Test Set: {test_f1}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'activation': 'relu', 'hidden_layer_sizes': (128, 64), 'learning_rate_init': 0.01, 'max_iter': 300, 'solver': 'adam'}
Best F1 Score (Train CV): 0.936839081173334
F1 Score on the Test Set: 0.07792207792207792


# Test 3

In [8]:
# Import necessary libraries
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# Resampling
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.99)
undersample = RandomUnderSampler(sampling_strategy=0.99, random_state=42)

# Create a resampling pipeline
resampling_pipeline = Pipeline([
    ('smote', smote),
    ('undersample', undersample)
])

# Resample the training data
X_train_balanced, y_train_balanced = resampling_pipeline.fit_resample(X_train, y_train)

# Feature scaling
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Define Stratified K-Fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Define the parameter grid for GridSearch
param_grid = {
    'hidden_layer_sizes': [(32, 16), (64, 32), (128, 64)],  # Neural network architectures
    'activation': ['relu', 'tanh'],  # Activation functions
    'solver': ['adam', 'sgd'],       # Optimizers
    'learning_rate_init': [0.001, 0.01],  # Initial learning rates
    'max_iter': [200, 300]           # Training iterations
}

# Define the model
mlp = MLPClassifier(random_state=42)

# Define the F1 scorer
f1_scorer = make_scorer(f1_score)

# Define GridSearchCV
grid_search = GridSearchCV(estimator=mlp, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Perform the grid search on the resampled data
grid_search.fit(X_train_balanced, y_train_balanced)

# Best parameters and best score
print(f"Best Parameters: {grid_search.best_params_}")
print(f"Best F1 Score (Train CV): {grid_search.best_score_}")

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print(f"F1 Score on the Test Set: {test_f1}")

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'activation': 'tanh', 'hidden_layer_sizes': (128, 64), 'learning_rate_init': 0.001, 'max_iter': 200, 'solver': 'adam'}
Best F1 Score (Train CV): 0.9759722406681961
F1 Score on the Test Set: 0.07058823529411765


c:\Users\aksha\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


# Test 4

In [9]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import make_scorer, f1_score, classification_report
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif

# Tip 1: Resampling (SMOTE + Undersampling Pipeline)
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.99)
undersample = RandomUnderSampler(sampling_strategy=0.99, random_state=42)
resampling_pipeline = Pipeline([
    ('smote', smote),
    ('undersample', undersample)
])

X_train_balanced, y_train_balanced = resampling_pipeline.fit_resample(X_train, y_train)

# Tip 2: Feature Scaling (RobustScaler handles outliers better than StandardScaler)
scaler = RobustScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)

# Tip 3: Dynamic Feature Selection
num_features = X_train.shape[1]
k_best_features = min(num_features, 10)  # Dynamically adjust for available features
feature_selector = SelectKBest(mutual_info_classif, k=k_best_features)
X_train_balanced = feature_selector.fit_transform(X_train_balanced, y_train_balanced)
X_test = feature_selector.transform(X_test)

# Tip 4: Stratified K-Fold for Stability
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Tip 5: Define Parameter Grid for GridSearch
param_grid = {
    'hidden_layer_sizes': [(64, 32), (128, 64, 32), (256, 128, 64)],  # Larger architectures
    'activation': ['relu', 'tanh'],  # Activation functions
    'solver': ['adam'],  # Use adam for stable convergence
    'learning_rate_init': [0.001, 0.005, 0.01],  # Learning rate
    'max_iter': [300, 500],  # Allow more training iterations
    'alpha': [0.0001, 0.001]  # L2 regularization for preventing overfitting
}

# Tip 6: Use a Custom F1 Scorer
f1_scorer = make_scorer(f1_score, average='binary')

# Tip 7: Use GridSearchCV for Hyperparameter Optimization
mlp = MLPClassifier(random_state=42)
grid_search = GridSearchCV(estimator=mlp, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Train the model
grid_search.fit(X_train_balanced, y_train_balanced)

# Display the best parameters and best training F1 score
print("Best Parameters:", grid_search.best_params_)
print("Best F1 Score (Train CV):", grid_search.best_score_)

# Tip 8: Test the model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print("\nF1 Score on the Test Set:", test_f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Tip 9 (Bonus): Save the model for future use
import joblib
joblib.dump(best_model, "best_mlp_model.pkl")


c:\Users\aksha\anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RobustScaler was fitted with feature names
  warnings.warn(


Fitting 5 folds for each of 72 candidates, totalling 360 fits
Best Parameters: {'activation': 'tanh', 'alpha': 0.0001, 'hidden_layer_sizes': (256, 128, 64), 'learning_rate_init': 0.001, 'max_iter': 300, 'solver': 'adam'}
Best F1 Score (Train CV): 0.9603167148538707

F1 Score on the Test Set: 0.04838709677419355

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.94      0.96      3246
           1       0.03      0.12      0.05        50

    accuracy                           0.93      3296
   macro avg       0.51      0.53      0.51      3296
weighted avg       0.97      0.93      0.95      3296



c:\Users\aksha\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


['best_mlp_model.pkl']

# Ultimate CODE

In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import BorderlineSMOTE, SMOTE
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.ensemble import VotingClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# Tip 1: Robust Data Scaling + Outlier Management
scaler = RobustScaler()

# PowerTransformer to normalize distributions (handles skewed data)
normalizer = PowerTransformer()

# Advanced feature engineering (if features < 10, skip PCA)
num_features = X_train.shape[1]
pca = PCA(n_components=min(num_features, 5)) if num_features > 10 else None

# Tip 2: Advanced Resampling (Hybrid SMOTE and Tomek Links)
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.99)
tomek = TomekLinks(sampling_strategy='majority')

resampling_pipeline = Pipeline([
    ('smote', smote),
    ('tomek', tomek)
])

# Resample the training data
X_train_balanced, y_train_balanced = resampling_pipeline.fit_resample(X_train, y_train)

# Apply Robust Scaling and Power Normalization
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_test = scaler.transform(X_test)
X_train_balanced = normalizer.fit_transform(X_train_balanced)
X_test = normalizer.transform(X_test)

# Tip 3: Feature Selection with Mutual Information
k_best_features = min(num_features, 10)
feature_selector = SelectKBest(mutual_info_classif, k=k_best_features)
X_train_balanced = feature_selector.fit_transform(X_train_balanced, y_train_balanced)
X_test = feature_selector.transform(X_test)

# Apply PCA if necessary
if pca:
    X_train_balanced = pca.fit_transform(X_train_balanced)
    X_test = pca.transform(X_test)

# Tip 4: Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Tip 5: Define Advanced Parameter Grids for Multiple Models
mlp_param_grid = {
    'hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'activation': ['relu', 'tanh'],
    'solver': ['adam'],
    'learning_rate_init': [0.001, 0.005],
    'alpha': [0.0001, 0.001],
    'max_iter': [500]
}

xgb_param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Tip 6: Train Multiple Models Using VotingClassifier
mlp = MLPClassifier(random_state=42)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
catboost = CatBoostClassifier(verbose=0, random_state=42)

# Define a Voting Classifier for ensemble
voting_clf = VotingClassifier(estimators=[
    ('mlp', mlp),
    ('xgb', xgb),
    ('cat', catboost)
], voting='soft')

# Create a pipeline to optimize all models
final_pipeline = Pipeline([
    ('model', voting_clf)
])

# Tip 7: Define a custom scorer for GridSearch
f1_scorer = make_scorer(f1_score, average='binary')

# Perform GridSearchCV to optimize VotingClassifier
param_grid = {
    'model__mlp__hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'model__mlp__alpha': [0.0001, 0.001],
    'model__xgb__n_estimators': [100, 200],
    'model__xgb__learning_rate': [0.01, 0.1],
    'model__xgb__max_depth': [3, 5],
    'model__cat__depth': [6, 8],
    'model__cat__learning_rate': [0.01, 0.1]
}

grid_search = GridSearchCV(estimator=final_pipeline, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Train the final model
grid_search.fit(X_train_balanced, y_train_balanced)

# Tip 8: Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print("\nBest Parameters:", grid_search.best_params_)
print("\nF1 Score on the Test Set:", test_f1)
print("\nClassification Report:")
print(classification_report(y_test, y_pred))

# Tip 9: Save the ultimate model
import joblib
joblib.dump(best_model, "ultimate_f1_model.pkl")


# Beyond ultimate

In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline


# Resampling techniques
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.99)
undersample = RandomUnderSampler(sampling_strategy=0.99, random_state=42)
tomek = TomekLinks(sampling_strategy='majority')

# Preprocessing steps
scaler = RobustScaler()
normalizer = PowerTransformer()

# Feature Selection
num_features = X_train.shape[1]
k_best = SelectKBest(mutual_info_classif, k=min(num_features, 10))

# Conditional PCA
pca = PCA(n_components=min(num_features, 10)) if num_features > 10 else None

# Models for ensemble learning
mlp = MLPClassifier(random_state=42, max_iter=1000)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
catboost = CatBoostClassifier(verbose=0, random_state=42)
rf_clf = RandomForestClassifier(random_state=42)
svc_clf = SVC(random_state=42, probability=True)

# Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('mlp', mlp),
    ('xgb', xgb),
    ('catboost', catboost),
    ('rf', rf_clf),
    ('svc', svc_clf)
], voting='soft')

# Pipeline definition
steps = [
    ('scaler', scaler),
    ('normalizer', normalizer),
    ('smote', smote),
    ('undersample', undersample),
    ('tomek', tomek),
    ('feature_selector', k_best),
]

# Add PCA if applicable
if pca:
    steps.append(('pca', pca))

# Add the model at the end of the pipeline
steps.append(('model', voting_clf))

# Create the pipeline
final_pipeline = Pipeline(steps)

# GridSearchCV Hyperparameter Tuning
param_grid = {
    'model__mlp__hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'model__mlp__alpha': [0.0001, 0.001],
    'model__xgb__n_estimators': [100, 200],
    'model__xgb__learning_rate': [0.01, 0.05],
    'model__xgb__max_depth': [3, 5],
    'model__catboost__depth': [6, 8],
    'model__catboost__learning_rate': [0.01, 0.05],
    'model__rf__n_estimators': [100, 300],
    'model__rf__max_depth': [10, 20],
    'model__svc__C': [1, 10],
    'model__svc__kernel': ['linear', 'rbf'],
    'model__svc__gamma': ['scale', 'auto']
}

# Custom F1 Scorer
f1_scorer = make_scorer(f1_score, average='binary')

# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(estimator=final_pipeline, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

# Output the best parameters and F1 score
print("\nBest Parameters:", grid_search.best_params_)
print("\nBest F1 Score (Train CV):", grid_search.best_score_)

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print("\nF1 Score on the Test Set:", test_f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Save the model
import joblib
joblib.dump(best_model, "ultimate_f1_model.pkl")


Fitting 10 folds for each of 4096 candidates, totalling 40960 fits


In [ ]:
# Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split
from sklearn.preprocessing import RobustScaler, PowerTransformer
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score, classification_report, make_scorer
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks
from imblearn.pipeline import Pipeline
import joblib

# Define the dataset split (assuming X_train, X_test, y_train, y_test are already defined)
# Example: X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42, test_size=0.2)

# Resampling techniques
smote = BorderlineSMOTE(random_state=42, sampling_strategy=0.99, n_jobs=-1)
undersample = RandomUnderSampler(sampling_strategy=0.99, random_state=42)
tomek = TomekLinks(sampling_strategy='majority', n_jobs=-1)

# Preprocessing steps
scaler = RobustScaler()
normalizer = PowerTransformer()

# Feature Selection
num_features = X_train.shape[1]
k_best = SelectKBest(mutual_info_classif, k=min(num_features, 10))

# Conditional PCA
pca = PCA(n_components=min(num_features, 10)) if num_features > 10 else None

# Models for ensemble learning
mlp = MLPClassifier(random_state=42, max_iter=1000)
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42, n_jobs=-1)
catboost = CatBoostClassifier(verbose=0, random_state=42, thread_count=-1)
rf_clf = RandomForestClassifier(random_state=42, n_jobs=-1)
svc_clf = SVC(random_state=42, probability=True)

# Voting Classifier
voting_clf = VotingClassifier(estimators=[
    ('mlp', mlp),
    ('xgb', xgb),
    ('catboost', catboost),
    ('rf', rf_clf),
    ('svc', svc_clf)
], voting='soft', n_jobs=-1)

# Pipeline definition
steps = [
    ('scaler', scaler),
    ('normalizer', normalizer),
    ('smote', smote),
    ('undersample', undersample),
    ('tomek', tomek),
    ('feature_selector', k_best),
]

# Add PCA if applicable
if pca:
    steps.append(('pca', pca))

# Add the model at the end of the pipeline
steps.append(('model', voting_clf))

# Create the pipeline
final_pipeline = Pipeline(steps)

# GridSearchCV Hyperparameter Tuning
param_grid = {
    'model__mlp__hidden_layer_sizes': [(128, 64, 32), (256, 128, 64)],
    'model__mlp__alpha': [0.0001, 0.001],
    'model__xgb__n_estimators': [100, 200],
    'model__xgb__learning_rate': [0.01, 0.05],
    'model__xgb__max_depth': [3, 5],
    'model__catboost__depth': [6, 8],
    'model__catboost__learning_rate': [0.01, 0.05],
    'model__rf__n_estimators': [100, 300],
    'model__rf__max_depth': [10, 20],
    'model__svc__C': [1, 10],
    'model__svc__kernel': ['linear', 'rbf'],
    'model__svc__gamma': ['scale', 'auto']
}

# Custom F1 Scorer
f1_scorer = make_scorer(f1_score, average='binary')

# Stratified K-Fold Cross-Validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# GridSearchCV
grid_search = GridSearchCV(estimator=final_pipeline, 
                           param_grid=param_grid, 
                           scoring=f1_scorer, 
                           cv=skf, 
                           verbose=2, 
                           n_jobs=-1)

# Fit the GridSearchCV
grid_search.fit(X_train, y_train)

# Output the best parameters and F1 score
print("\nBest Parameters:", grid_search.best_params_)
print("\nBest F1 Score (Train CV):", grid_search.best_score_)

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
test_f1 = f1_score(y_test, y_pred)

print("\nF1 Score on the Test Set:", test_f1)
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Save the model
joblib.dump(best_model, "ultimate_f1_model.pkl")
